In [1]:
import msticpy as mp
mp.init_notebook()
# qry_prov = mp.QueryProvider("MSSentinel_New")
# qry_prov.connect()
import sys
sys.path.append(r'../src')
%load_ext autoreload
%autoreload 2

E:\src\msticpy\msticpy\transform\IoCExtract.py:669: UserWarning: registration of accessor <class 'msticpy.transform.IoCExtract.IoCExtractAccessor'> under name 'mp_ioc' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  @pd.api.extensions.register_dataframe_accessor("mp_ioc")


In [5]:
!dir ..\..\notebooks\demos\data

 Volume in drive E has no label.
 Volume Serial Number is 7E50-19F7

 Directory of e:\src\notebooks\demos\data

2022-01-24  19:36    <DIR>          .
2023-07-24  15:58    <DIR>          ..
2020-02-13  09:53            58,327 alerts_list.pkl
2019-12-05  15:11        18,391,639 all_events_df.pkl
2019-11-02  12:24           385,203 az_net_comms_df.pkl
2019-12-05  15:11            10,735 az_net_df.pkl
2019-12-05  15:11           180,577 clus_events.pkl
2019-12-05  15:11             6,560 clus_logons.pkl
2019-12-05  15:11             5,362 combined.pkl
2019-12-05  15:11            16,737 dest_pos.pkl
2019-12-05  15:11             4,920 event_pivot.pkl
2019-12-05  15:11             2,369 event_subset.pkl
2019-12-05  15:11             2,614 failedLogons.pkl
2019-12-05  15:11        11,044,021 feature_procs.pkl
2019-11-02  12:24            70,859 flows_df.pkl
2019-12-05  15:11           177,780 flow_index.pkl
2019-12-05  15:11             3,837 flow_sum_df.pkl
2019-12-05  15:11             2,7

In [2]:
win_logons_df = pd.read_pickle("../../notebooks/demos/data/host_logons.pkl")
win_procs_df = pd.read_pickle("../../notebooks/demos/data/processes_on_host.pkl")
lx_procs_df = pd.read_pickle("../../notebooks/demos/data/linux_proc_test.pkl")

In [24]:
import yaml
from pathlib import Path

entities = Path("../src/entities.yaml").read_text()
df_prompts = yaml.safe_load(Path("../src/df_prompts.yaml").read_text())

In [15]:
df_prompts["prompts"].keys()

dict_keys(['chain_tasks', 'df_generic_task', 'classify_df', 'df_process_tree_schema', 'df_identify_entities'])

In [14]:
print(df_prompts["prompts"]["df_generic_task"])

You are an assistant that is expert in CyberSecurity concepts.
You are provided with a pandas dataframe (df) with {num_rows} rows and {num_columns} columns.
This is the result of `print(df.head({rows_to_display}))`:
{df_head}.

When asked about the data, do the following:
{task}



In [4]:
from pd_queries import analyze_df

In [6]:
# classify_prompt = df_prompts["prompts"]["df_generic_task"].format(task=df_prompts["prompts"]["classify_df"])
analyze_df(win_logons_df, df_prompts["prompts"]["df_generic_task"]["prompt"], df_prompts["prompts"]["df_classify"]["prompt"])

'{\n  "inferred data type": "logon/authentication events",\n  "inferred data origin": "windows host"\n}'

In [7]:
print(analyze_df(lx_procs_df, df_prompts["prompts"]["df_generic_task"]["prompt"], df_prompts["prompts"]["df_process_tree_schema"]["prompt"]))

{
  "process_name": "exe",
  "process_id": "pid",
  "parent_id": "ppid",
  "time_stamp": "TimeGenerated",
  "cmd_line": "cmdline",
  "user_name": null,
  "logon_id": null,
  "host_name": null,
  "parent_name": null,
  "target_logon_id": null,
  "user_id": null,
  "event_id": null
}


In [8]:
entity_task = df_prompts["prompts"]["df_identify_entities"]["prompt"].format(ontology_entities=entities)
print(analyze_df(win_logons_df, df_prompts["prompts"]["df_generic_task"]["prompt"], entity_task))



{
  "TenantId": "Account.AAD_USER_PRINCIPAL_NAME",
  "Account": "Account.WINDOWS_USER_NAME",
  "EventID": null,
  "TimeGenerated": "TIME_STAMP",
  "SourceComputerId": null,
  "Computer": "HOST_NAME",
  "SubjectUserName": "Account.WINDOWS_USER_NAME",
  "SubjectDomainName": "DOMAIN_NAME",
  "SubjectUserSid": null,
  "TargetUserName": "Account.WINDOWS_USER_NAME",
  "TargetDomainName": "DOMAIN_NAME",
  "TargetUserSid": null,
  "TargetLogonId": null,
  "LogonProcessName": "PROCESS_NAME",
  "LogonType": null,
  "AuthenticationPackageName": null,
  "Status": null,
  "IpAddress": "IP_ADDRESS",
  "WorkstationName": "HOST_NAME",
  "TimeCreatedUtc": "TIME_STAMP"
}


In [14]:
print(analyze_df(win_procs_df, df_prompts["prompts"]["df_generic_task"]["prompt"], entity_task))

{
  "TenantId": "Account.AAD_USER_PRINCIPAL_NAME",
  "Account": "Account.WINDOWS_USER_NAME",
  "EventID": null,
  "TimeGenerated": "TIME_STAMP",
  "Computer": "HOST_NAME",
  "SubjectUserSid": null,
  "SubjectUserName": "Account.WINDOWS_USER_NAME",
  "SubjectDomainName": null,
  "SubjectLogonId": null,
  "NewProcessId": null,
  "NewProcessName": "FILE_PATH",
  "TokenElevationType": null,
  "ProcessId": null,
  "CommandLine": "COMMAND_LINE",
  "ParentProcessName": "PROCESS_NAME",
  "TargetLogonId": null,
  "SourceComputerId": null,
  "TimeCreatedUtc": "TIME_STAMP"
}


In [19]:
print(analyze_df(lx_procs_df, df_prompts["prompts"]["df_generic_task"]["prompt"], entity_task))

{
  "EventType": "null.null",
  "TenantId": "null.null",
  "Computer": "Host.HOST_NAME",
  "mssg_id": "null.null",
  "TimeGenerated": "null.null",
  "a0": "null.null",
  "a1": "null.null",
  "a2": "null.null",
  "argc": "null.null",
  "auid": "null.null",
  "cmdline": "Process.COMMAND_LINE",
  "cwd": "File.FILE_PATH",
  "egid": "null.null",
  "euid": "null.null",
  "exe": "File.FILE_NAME",
  "gid": "null.null",
  "pid": "null.null",
  "ppid": "null.null",
  "proctitle": "null.null",
  "ses": "null.null",
  "success": "null.null",
  "uid": "null.null"
}


In [9]:
 discarded = """If are other columns not defined here, e.g. "UserDisplayName" that relate
  directly to the account (i.e. they have the term "account" or "user" in the name),
  add the following attribute to the account including those columns names.
  (don't include columns that are already mapped to one of the above attributes):
  - RelatedColumns (list of strings)
"""

find_accounts_prompt = """
Your task is to map the columns of the source dataframe that refer
to computer accounts. You *must* return the results that you find in
the source dataframe, not code to find them.

1. Identify the columns that refer to accounts.
  Accounts will have at minimum this attribute:
  - Name (string) - the name of the account (synonyms: account name, account, user, username, user principal name, upn)
    The name may be prefixed with a string identifying its role in the event, e.g. "TargetUserName", "SubjectUserName"

  Accounts may also have these additional attributes:
  - Domain (string) - the name of the account (synonyms: account name, account, user, user principal name, upn)
  - SID (string) - the security identifier of the account (synonyms: sid)
  - UserID (string) - the integer user ID of the account (synonyms: userid, uid, euid). There may be multiple user IDs for the same account.
  - ObjectGUID (string) - the object UUID of the account (synonyms: objectguid, AADUserID)
  - LogonID (string) - the logon ID of the account (synonyms: logonid, sess, sessionid)

2. Identify the role of each account in the event.
  Multiple accounts may be present in the same event, e.g. "TargetUserName", "SubjectUserName".
  These represent different roles in the event, e.g. the target account and the subject account.
  The column names of each account role will typically have the same prefix.
  Try to identify the role of each account in the event, and group the accounts by role and
  add the following attribute to each account:
  - Role (string) - the role of the account in the event.
  Do not remove the role prefix from the column name that you return in the answer.

3. Return the results.

  You *must* return the results that you found by analyzing the source dataframe, do not
  return code to find them.

  Return the answer as a JSON list of dictionaries, where each dictionary represents the
  mapping between the above attributes and a column found in the data, grouped by account
  role (if any). Use the attribute names defined above as the dictionary keys and the mapped
  column name as values. If you find no suitable mapping for an attribute, return null as the value
  for that attribute.

  Do not include duplicate definitions of the same account dictionary in the answer.

  If you have at least one account-column mapping to return, discard any other
  account mappings that only have Name and RelatedColumns that are not null.
"""
result = analyze_df(win_logons_df, df_prompts["prompts"]["df_generic_task"]["prompt"], task=find_accounts_prompt, sample_rows=5, max_tokens=1000)
print(result)

[
  {
    "Name": "SubjectUserName",
    "Domain": "SubjectDomainName",
    "SID": "SubjectUserSid",
    "UserID": null,
    "ObjectGUID": null,
    "LogonID": null,
    "Role": "Subject"
  },
  {
    "Name": "TargetUserName",
    "Domain": "TargetDomainName",
    "SID": "TargetUserSid",
    "UserID": null,
    "ObjectGUID": null,
    "LogonID": "TargetLogonId",
    "Role": "Target"
  },
  {
    "Name": "Account",
    "Domain": null,
    "SID": null,
    "UserID": null,
    "ObjectGUID": null,
    "LogonID": null,
    "Role": null
  }
]


In [52]:
result = analyze_df(win_procs_df, df_prompts["prompts"]["df_generic_task"], task=find_accounts_prompt, sample_rows=5, max_tokens=1000)
print(result)

[
  {
    "Name": "SubjectUserName",
    "Domain": "SubjectDomainName",
    "SID": "SubjectUserSid",
    "UserID": null,
    "ObjectGUID": null,
    "LogonID": "SubjectLogonId",
    "Role": "Subject"
  },
  {
    "Name": "TargetUserName",
    "Domain": null,
    "SID": null,
    "UserID": null,
    "ObjectGUID": null,
    "LogonID": "TargetLogonId",
    "Role": "Target"
  },
  {
    "Name": "Account",
    "Domain": null,
    "SID": null,
    "UserID": null,
    "ObjectGUID": null,
    "LogonID": null,
    "Role": null
  }
]


In [32]:
mde_proc_df = pd.read_csv("e:/src/msticpy/tests/testdata/mde_proc_cs1.csv")
mde_proc_pub_df = pd.read_pickle("e:/src/msticpy/tests/testdata/mde_proc_pub.pkl")
result = analyze_df(mde_proc_df, df_prompts["prompts"]["df_generic_task"]["prompt"], task=find_accounts_prompt, sample_rows=2, max_tokens=1000)
print(result)
result = analyze_df(mde_proc_pub_df, df_prompts["prompts"]["df_generic_task"]["prompt"], task=find_accounts_prompt, sample_rows=2, max_tokens=1000)
print(result)

[
  {
    "Name": "CreatedProcessAccountName",
    "Domain": "CreatedProcessAccountDomainName",
    "SID": "CreatedProcessAccountSid",
    "UserID": null,
    "ObjectGUID": null,
    "LogonID": null,
    "Role": "CreatedProcess"
  },
  {
    "Name": "InitiatingProcessAccountName",
    "Domain": "InitiatingProcessAccountDomainName",
    "SID": "InitiatingProcessAccountSid",
    "UserID": null,
    "ObjectGUID": null,
    "LogonID": null,
    "Role": "InitiatingProcess"
  }
]
[
  {
    "Name": "AccountName",
    "Domain": "AccountDomain",
    "SID": "AccountSid",
    "UserID": null,
    "ObjectGUID": null,
    "LogonID": null,
    "Role": "Subject"
  },
  {
    "Name": "TargetUserName",
    "Domain": "TargetDomainName",
    "SID": "TargetUserSid",
    "UserID": null,
    "ObjectGUID": null,
    "LogonID": null,
    "Role": "Target"
  },
  {
    "Name": "InitiatingProcessAccountName",
    "Domain": "InitiatingProcessAccountDomain",
    "SID": "InitiatingProcessAccountSid",
    "UserID": n

In [45]:
 discarded = """If are other columns not defined here, e.g. "UserDisplayName" that relate
  directly to the account (i.e. they have the term "account" or "user" in the name),
  add the following attribute to the account including those columns names.
  (don't include columns that are already mapped to one of the above attributes):
  - RelatedColumns (list of strings)

  Example responses:
    Two columns "SourceIP" and "DestIP" found IP addresses found
        [
            { "Address": "SourceIP", "Role": "Source", "RelatedColumns": ["SourcePort"] },
            { "Address": "DestIP", "Role": "Dest", "RelatedColumns": [] },
        ]
    Single IP address column found
        [
            { "Address": "IpAddress", "Role": null, "RelatedColumns": [] },
        ]

   In all cases, the value of the "Address" attribute
   must the name of column from the source dataframe that has been
   identified as an IP address column - do not make up values for your answers or add
   any other columns.

   Ensure that you follow these rules for your answer:
   - 
   - Do not include columns in your answer that are not present in the source dataframe.
   - Do not include duplicate definitions of the same item in your answer.

"""

find_ips_prompt = """

Instructions
------------
Overview: You are asked follow the instructions below
to identify which columns of the source dataframe 
(enclosed in triple backticks) contain IP Addresses and return the
answer as a JSON list. Do not return code as your answer - return
a JSON dictionary as described in item 2 with the result of
*your* analysis of the data.

1. Identify which columns in the source dataframe (if any) contain IP addresses.
   Use the context above together with both the source dataframe *data values*
   and *column names* (headers) to identify which columns (if any) could contain IP Addresses.
   - A column name containing an IP address will almost always contain the term "IP"
   - Common synonyms for IP address are "IP", "IPAddress", "IPAddr", "IPAddress"
   - IP addresses can be either a version 4 (PV4) dot-delimited string
    (e.g. "52.138.20.1", "20.10.10.1") or a version 6 (IPV6) colon-delimited
    string (e.g. "2001:0db8:85a3:0000:0000:8a2e:0370:7334",
    "2001:db8:85a3:0:0:8a2e:370:7334", "::1")

2. For each IP address column that you find in the source dataframe,
   record the following attributes:
    - column_name
      (string) - the name of the column from the source dataframe
      that contains IP address data.
      The value of this attribute *must* be the name of a column in the source dataframe
      ()
      (do not make up values for your answers or add any other columns)
    - role
      (optional string) - the value is the role (if any) of the IP address column.
      IP address column names are commonly prefixed with a role, e.g. "SourceIP"
      (role = "Source"), "DestIPAddress" (role = "Dest")
      (especially where multiple IP address columns are present in the source dataframe)
      If there is only one IP address column in the source dataframe, then roles
      may not applicable, so set this to null.
    - related_columns
      (list of strings) - If there are multiple IP columns in the data, other
      (non-IP) may be related to these columns if they share the same prefix or suffix as the IP address column.
      e.g. "SourcePort" column is related to the "SourceIP" IP column via the shared
      prefix "Source".

3. Return the results of your analysis as a JSON array of objects, one object per IP address column.

  The output should have the following format:
      [
          { "Address": <column_name>, "Role": <role|null>, "RelatedColumns": [<related_columns>] },
          ...
      ]

4. Do not return code as your answer - return the JSON dictionary as described in item 2.

"""
result = analyze_df(win_logons_df, df_prompts["prompts"]["df_generic_task"]["prompt"], task=find_ips_prompt, sample_rows=1, max_tokens=1000)
print(result)

{
    "results": [
        {
            "Address": "IpAddress",
            "Role": null,
            "RelatedColumns": ["SourceComputerId", "Computer", "IpAddress", "WorkstationName"]
        },
        {
            "Address": "IpAddress2",
            "Role": null,
            "RelatedColumns": ["IpAddress2", "IpAddress3"]
        }
    ]
}


In [153]:
find_ips_prompt = """

Instructions
------------
Overview: You are asked follow the instructions below
to identify which columns of the source dataframe 
(enclosed in triple backticks) contain IP Addresses and return the
answer as a JSON list. Do not return code as your answer - return
a JSON dictionary as described in item 2 with the result of
*your* analysis of the data.

1. Identify which columns in the source dataframe (if any) contain IP addresses.
   Use the context above together with both the source dataframe *data values*
   and *column names* (headers) to identify which columns (if any) could contain IP Addresses.
   - A column name containing an IP address will almost always contain the term "IP"
   - Common synonyms for IP address are "IP", "IPAddress", "IPAddr", "IPAddress"
   - IP addresses can be either a version 4 (PV4) dot-delimited string
    (e.g. "52.138.20.1", "20.10.10.1") or a version 6 (IPV6) colon-delimited
    string (e.g. "2001:0db8:85a3:0000:0000:8a2e:0370:7334",
    "2001:db8:85a3:0:0:8a2e:370:7334", "::1")

2. For each IP address column that you find in the source dataframe,
   record the following attributes:
    - column_name
      (string) - the name of the column from the source dataframe
      that contains IP address data.
      The value of this attribute *must* be the name of a column in the source dataframe
      ()
      (do not make up values for your answers or add any other columns)
    - role
      (optional string) - the value is the role (if any) of the IP address column.
      IP address column names are commonly prefixed with a role, e.g. "SourceIP"
      (role = "Source"), "DestIPAddress" (role = "Dest")
      (especially where multiple IP address columns are present in the source dataframe)
      If there is only one IP address column in the source dataframe, then roles
      may not applicable, so set this to null.
    - related_columns
      (list of strings) - If there are multiple IP columns in the data, other
      (non-IP) may be related to these columns if they share the same prefix or suffix as the IP address column.
      e.g. "SourcePort" column is related to the "SourceIP" IP column via the shared
      prefix "Source".

3. Return the results of your analysis as a JSON array of objects, one object per IP address column.

  The output should have the following format:
      [
          { "Address": <column_name>, "Role": <role|null>, "RelatedColumns": [<related_columns>] },
          ...
      ]

4. Do not return code as your answer - return the JSON dictionary as described in item 2.

"""

find_ip_cols_prompt = """
You are asked to identify which columns of the Source dataframe 
(enclosed in triple backticks) contain IP Addresses.

Use your knowledge of common synonyms for IP addresses
use the column names (the headers on the first row of "Source dataframe")
to identify which columns (if any) contain IP Addresses.
The data might not contain any IP addresses, so you should return an empty list if that is the case.
If you are doubtful whether about a column is IP address column, do not
return it as an answer. Do not include columns with similarly
spelled common terms such as "Up" and "Id" unless also they explicitly
include the term "IP" in the column name.

Return the answer as a JSON list. Do not include an explanation of your
reasoning or any code as your answer - return a JSON list of unique column names.

If you find no IP address columns, return an empty list.
"""

# find_ip_cols_prompt = """
# You are an assistant that is expert in CyberSecurity concepts.
#   You are provided with a pandas dataframe (df) with {num_rows} rows and {num_columns} columns.
#   This is a sample of the source dataframe (enclosed in triple backticks).

#   Source dataframe:
#   ```{df_head}```

#   When asked about the data, do the following:
# """
result = analyze_df(win_logons_df, df_prompts["prompts"]["df_generic_task"]["prompt"], task=find_ip_cols_prompt, sample_rows=1, max_tokens=1000)
print(result)

["IpAddress"]


In [145]:
result = analyze_df(net_flows_df, df_prompts["prompts"]["df_generic_task"]["prompt"], task=find_ip_cols_prompt, sample_rows=1, max_tokens=1000)
print(result)

["VMIPAddress", "PublicIPs", "SrcIP", "DestIP", "AllExtIPs"]


In [141]:
mde_proc_pub_df.head(0)

,Timestamp,DeviceId,DeviceName,ActionType,FileName,FolderPath,SHA1,SHA256,MD5,FileSize,ProcessVersionInfoCompanyName,ProcessVersionInfoProductName,ProcessVersionInfoProductVersion,ProcessVersionInfoInternalFileName,ProcessVersionInfoOriginalFileName,ProcessVersionInfoFileDescription,ProcessId,ProcessCommandLine,ProcessIntegrityLevel,ProcessTokenElevation,ProcessCreationTime,AccountDomain,AccountName,AccountSid,AccountUpn,...,InitiatingProcessTokenElevation,InitiatingProcessSHA1,InitiatingProcessSHA256,InitiatingProcessMD5,InitiatingProcessFileName,InitiatingProcessFileSize,InitiatingProcessVersionInfoCompanyName,InitiatingProcessVersionInfoProductName,InitiatingProcessVersionInfoProductVersion,InitiatingProcessVersionInfoInternalFileName,InitiatingProcessVersionInfoOriginalFileName,InitiatingProcessVersionInfoFileDescription,InitiatingProcessId,InitiatingProcessCommandLine,InitiatingProcessCreationTime,InitiatingProcessFolderPath,InitiatingProcessParentId,InitiatingProcessParentFileName,InitiatingProcessParentCreationTime,InitiatingProcessSignerType,InitiatingProcessSignatureStatus,ReportId,AppGuardContainerId,AdditionalFields,CreatedProcessParentId


In [154]:
result = analyze_df(mde_proc_pub_df, df_prompts["prompts"]["df_generic_task"]["prompt"], task=find_ip_cols_prompt, sample_rows=0, max_tokens=1000)
print(result)

["AccountUpn", "InitiatingProcessAccountUpn"]


In [48]:
print(win_logons_df.head(5).to_json(orient="records"))

[{"TenantId":"52b1ab41-869e-4138-9e40-2a4457f09bf0","Account":"NT AUTHORITY\\SYSTEM","EventID":4624,"TimeGenerated":1549947394307,"SourceComputerId":"263a788b-6526-4cdc-8ed9-d79402fe4aa0","Computer":"MSTICAlertsWin1","SubjectUserName":"MSTICAlertsWin1$","SubjectDomainName":"WORKGROUP","SubjectUserSid":"S-1-5-18","TargetUserName":"SYSTEM","TargetDomainName":"NT AUTHORITY","TargetUserSid":"S-1-5-18","TargetLogonId":"0x3e7","LogonProcessName":"Advapi  ","LogonType":5,"AuthenticationPackageName":"Negotiate","Status":"","IpAddress":"-","WorkstationName":"-","TimeCreatedUtc":1549947394307},{"TenantId":"52b1ab41-869e-4138-9e40-2a4457f09bf0","Account":"MSTICAlertsWin1\\MSTICAdmin","EventID":4624,"TimeGenerated":1549946245340,"SourceComputerId":"263a788b-6526-4cdc-8ed9-d79402fe4aa0","Computer":"MSTICAlertsWin1","SubjectUserName":"-","SubjectDomainName":"-","SubjectUserSid":"S-1-0-0","TargetUserName":"MSTICAdmin","TargetDomainName":"MSTICAlertsWin1","TargetUserSid":"S-1-5-21-996632719-2361334927

In [137]:
mde_proc_pub_df.columns

Index(['Timestamp', 'DeviceId', 'DeviceName', 'ActionType', 'FileName',
       'FolderPath', 'SHA1', 'SHA256', 'MD5', 'FileSize',
       'ProcessVersionInfoCompanyName', 'ProcessVersionInfoProductName',
       'ProcessVersionInfoProductVersion',
       'ProcessVersionInfoInternalFileName',
       'ProcessVersionInfoOriginalFileName',
       'ProcessVersionInfoFileDescription', 'ProcessId', 'ProcessCommandLine',
       'ProcessIntegrityLevel', 'ProcessTokenElevation', 'ProcessCreationTime',
       'AccountDomain', 'AccountName', 'AccountSid', 'AccountUpn',
       'AccountObjectId', 'LogonId', 'InitiatingProcessAccountDomain',
       'InitiatingProcessAccountName', 'InitiatingProcessAccountSid',
       'InitiatingProcessAccountUpn', 'InitiatingProcessAccountObjectId',
       'InitiatingProcessLogonId', 'InitiatingProcessIntegrityLevel',
       'InitiatingProcessTokenElevation', 'InitiatingProcessSHA1',
       'InitiatingProcessSHA256', 'InitiatingProcessMD5',
       'InitiatingProcessFil

In [134]:
result = analyze_df(mde_proc_pub_df, df_prompts["prompts"]["df_generic_task"]["prompt"], task=find_ip_cols_prompt, sample_rows=2, max_tokens=1000)
print(result)

["IpAddress", "IP", "IPAddr", "IPAddress", "IP_Address", "IP_Address_1", "IP_Address_2", "IP_Address_3", "IP_Address_4", "IP_Address_5", "IP_Address_6", "IP_Address_7", "IP_Address_8", "IP_Address_9", "IP_Address_10", "IP_Address_11", "IP_Address_12", "IP_Address_13", "IP_Address_14", "IP_Address_15", "IP_Address_16", "IP_Address_17", "IP_Address_18", "IP_Address_19", "IP_Address_20", "IP_Address_21", "IP_Address_22", "IP_Address_23", "IP_Address_24", "IP_Address_25", "IP_Address_26", "IP_Address_27", "IP_Address_28", "IP_Address_29", "IP_Address_30", "IP_Address_31", "IP_Address_32", "IP_Address_33", "IP_Address_34", "IP_Address_35", "IP_Address_36", "IP_Address_37", "IP_Address_38", "IP_Address_39", "IP_Address_40", "IP_Address_41", "IP_Address_42", "IP_Address_43", "IP_Address_44", "IP_Address_45", "IP_Address_46", "IP_Address_47", "IP_Address_48", "IP_Address_49", "IP_Address_50"]


In [130]:
ip_prompt2 = """
IP column definitions (enclosed in "```" and "```")
```
- type_name: IP_ADDRESS
  description: An IP version 4 address
  entity: IpAddress
  examples:
    - 52.138.20.1
    - 20.10.10.1
- type_name: IPV6_ADDRESS
  description: An IP version 6 address
  entity: IpAddress
  examples:
    - 2001:0db8:85a3:0000:0000:8a2e:0370:7334
    - 2001:db8:85a3:0:0:8a2e:370:7334
```
Sample data (enclosed in "<<<" and ">>>")
<<<
{data}
>>>

You are asked to analyse the data in "Sample data" section above (enclosed in "<<<" and ">>>")
and identity which columns contain IP addresses (an IP Column) - as
defined in "IP column definitions" (enclosed in triple backticks) and return the results
as a JSON dictionary.

The first line of "Sample data" contains the column names
and subsequent lines are sample data values for each column.

1. Using the definitions given in "IP Column Definitions", examine the column names
   and column data given in "Sample data".
   If the data pattern in the "Sample data" for a column matches the definition examples,
   this is a positive match as an IP column.
   
2. Create a list of the column names that are positively matched as IP columns in step 1.

3. From the subset of columns identified in step 2:
  - If the IP column name has a qualifying prefix 
    (e.g. "src_ip" - prefix "src", "DestIpAddress" - prefix "Dest", "example_ip_addr" - prefix "example") 
    or a qualifying suffix 
    (e.g. "ip_source" - suffix "source" or "ip_external" - suffix "external", "ip_address_example" - suffix "example")
    then extract the prefix or suffix from the column name
    and add it as the value of a "role" key in the JSON dictionary entry for that column.
  - If it has no prefix or suffix, do not assign the column a "role"

4. Return a JSON dictionary with the following structure:
  {{ <column_name>: {{ "role": <role> }}}}
  Only include a column name once in the output.
  Do not return code as a solution.

5. After the dictionary is returned, say, for each column, why it was identified as an IP column.
"""
# For each column in the results add a "reason" item to the dictionary entry with
# and explanation of why the column was identified as an IP column.

# Use the following examples to test your solution:
# Examples:
# ---------
# Example 1:
#   data:
#   "                              TenantId                     Account  EventID           TimeGenerated                      SourceComputerId
#   0  52b1ab41-869e-4138-9e40-2a4457f09bf0         NT AUTHORITY\SYSTEM     4624 2019-02-12 04:56:34.307  263a788b-6526-4cdc-8ed9-d79402fe4aa0"
#   Expected results:
#   {{}}
# Example 2:
#   data:
#   "                              TenantId           RemoteIP  EventID           TimeGenerated                      SourceComputerId
#   0  52b1ab41-869e-4138-9e40-2a4457f09bf0         124.43.5.3     4624 2019-02-12 04:56:34.307  263a788b-6526-4cdc-8ed9-d79402fe4aa0"
#   Expected results:
#   {{ "RemoteIP": {{ "role": "Remote" }} }}
# Example 3:
#   data:
#   "                              TenantId  ListeningAddress  EventID           ListeningPort                      SourceComputerId
#   0  52b1ab41-869e-4138-9e40-2a4457f09bf0         127.0.0.1     4624                    8080  263a788b-6526-4cdc-8ed9-d79402fe4aa0"
#   Expected results:
#   {{ "ListeningAddress": {{ "role": "Listening", "related_columns": ["ListeningPort"] }} }}

# """
  #  If there is no matching sample data, only positively identify a column as
  #  an IP column if its name contains the substring "IP" and the name
  #  appears to match the definition for IP addresses given in the IP Column definitions.

  # return this list as an intermediate result in the JSON dictionary.
  #  Add an explanation for each column of why they were identified as IP columns.


  # If there is no matching sample data, tentatively identify the "Sample data" column as
  #  an IP column only if the substring "IP" appears in its name and the name
  #  appears to be a synonym of one of the IP Column definitions.
  #  Do not include any columns that are not present in the "Sample data" column names.

from pd_queries import get_completion

get_completion(ip_prompt2.format(data=win_logons_df.head(5).to_string()), model="gpt-3.5-turbo", max_tokens=1000, temperature=0)

'{\n  "IpAddress": {},\n  "WorkstationName": {},\n  "SourceComputerId": {},\n  "TargetUserName": {},\n  "TargetDomainName": {},\n  "TargetUserSid": {},\n  "TargetLogonId": {},\n  "IpAddress": {},\n  "WorkstationName": {}\n}\n\nThe columns identified as IP columns are:\n- IpAddress: The data pattern in the "Sample data" matches the examples given in the IP column definition for IP_ADDRESS.\n- SourceComputerId: The data pattern in the "Sample data" matches the examples given in the IP column definition for IP_ADDRESS.\n- TargetUserName: The data pattern in the "Sample data" matches the examples given in the IP column definition for IP_ADDRESS.\n- TargetDomainName: The data pattern in the "Sample data" matches the examples given in the IP column definition for IP_ADDRESS.\n- TargetUserSid: The data pattern in the "Sample data" matches the examples given in the IP column definition for IP_ADDRESS.\n- TargetLogonId: The data pattern in the "Sample data" matches the examples given in the IP 

In [112]:
print(ip_prompt2.format(data=win_logons_df.head(5).to_string()))


IP column definitions (enclosed in "```" and "```")
```
- type_name: IP_ADDRESS
    description: An IP version 4 address
    entity: IpAddress
    examples:
      - 52.138.20.1
      - 20.10.10.1
  - type_name: IPV6_ADDRESS
    description: An IP V6 address
    entity: IpAddress
    examples:
        - 2001:0db8:85a3:0000:0000:8a2e:0370:7334
        - 2001:db8:85a3:0:0:8a2e:370:7334
```
Sample data (enclosed in "<<<" and ">>>")
<<<
                               TenantId                     Account  EventID           TimeGenerated                      SourceComputerId         Computer   SubjectUserName SubjectDomainName SubjectUserSid TargetUserName TargetDomainName                                 TargetUserSid TargetLogonId LogonProcessName  LogonType AuthenticationPackageName Status        IpAddress WorkstationName          TimeCreatedUtc
0  52b1ab41-869e-4138-9e40-2a4457f09bf0         NT AUTHORITY\SYSTEM     4624 2019-02-12 04:56:34.307  263a788b-6526-4cdc-8ed9-d79402fe4aa0  MSTICA

In [129]:
net_flows_df = pd.read_csv("e:/src/msticpy/tests/testdata/az_net_flows.csv")
result = get_completion(ip_prompt2.format(data=net_flows_df.head(5)), model="gpt-3.5-turbo", max_tokens=1000, temperature=0)
print(result)

{
  "VMIPAddress": {},
  "PublicIPs": {},
  "SrcIP": {},
  "DestIP": {},
  "AllExtIPs": {}
}


In [135]:
net_flows_df.iloc[0]

Unnamed: 0                                              881
TenantId               52b1ab41-869e-4138-9e40-2a4457f09bf0
TimeGenerated                       2019-02-12 14:22:40.697
FlowStartTime                       2019-02-12 13:00:07.000
FlowEndTime                         2019-02-12 13:45:08.000
FlowIntervalEndTime                 2019-02-12 14:00:00.000
FlowType                                        AzurePublic
ResourceGroup                         asihuntomsworkspacerg
VMName                                      msticalertswin1
VMIPAddress                                        10.0.3.5
PublicIPs                                  ['65.55.44.109']
SrcIP                                                   NaN
DestIP                                                  NaN
L4Protocol                                                T
L7Protocol                                            https
DestPort                                              443.0
FlowDirection                           